# A - Advanced Construction

## Setup

If you haven't already, install the toolkit and dependencies using the [Setup](./00-Setup.ipynb) notebook.

## Extract and build pipelines

See [Advanced graph construction](https://github.com/awslabs/graphrag-toolkit/blob/main/docs/lexical-graph/indexing.md#advanced-graph-construction).

In [3]:
%reload_ext dotenv
%dotenv

import os

from graphrag_toolkit.lexical_graph.tenant_id import TenantId
from graphrag_toolkit.lexical_graph.storage import GraphStoreFactory
from graphrag_toolkit.lexical_graph.storage import VectorStoreFactory
from graphrag_toolkit.lexical_graph.indexing import sink
from graphrag_toolkit.lexical_graph.indexing.constants import PROPOSITIONS_KEY, DEFAULT_ENTITY_CLASSIFICATIONS
from graphrag_toolkit.lexical_graph.indexing.extract import LLMPropositionExtractor
from graphrag_toolkit.lexical_graph.indexing.extract import TopicExtractor
from graphrag_toolkit.lexical_graph.indexing.extract import GraphScopedValueStore
from graphrag_toolkit.lexical_graph.indexing.extract import ScopedValueProvider, DEFAULT_SCOPE
from graphrag_toolkit.lexical_graph.indexing.extract import ExtractionPipeline
from graphrag_toolkit.lexical_graph.indexing.build import Checkpoint
from graphrag_toolkit.lexical_graph.indexing.build import BuildPipeline
from graphrag_toolkit.lexical_graph.indexing.build import VectorIndexing
from graphrag_toolkit.lexical_graph.indexing.build import GraphConstruction
from graphrag_toolkit.lexical_graph.storage.graph import MultiTenantGraphStore
from graphrag_toolkit.lexical_graph.storage.vector import MultiTenantVectorStore


from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.web import SimpleWebPageReader

checkpoint = Checkpoint('multi-tenant-advanced-construction-example', enabled=True)

tenant_id = TenantId('tenant1')

# Create graph and vector stores

graph_store = MultiTenantGraphStore.wrap(
    GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE']),
    tenant_id=tenant_id
)

vector_store = MultiTenantVectorStore.wrap(
    VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE']),
    tenant_id=tenant_id
)

# Create extraction pipeline components

 # 1. Chunking using SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=256,
    chunk_overlap=20)

# 2. Proposition extraction

proposition_extractor = LLMPropositionExtractor()

# 3. Topic extraction

entity_classification_provider = ScopedValueProvider(
    label='EntityClassification',
    scoped_value_store=GraphScopedValueStore(graph_store=graph_store),
    initial_scoped_values = { DEFAULT_SCOPE: DEFAULT_ENTITY_CLASSIFICATIONS }
)

topic_extractor = TopicExtractor(
    source_metadata_field=PROPOSITIONS_KEY, # Omit this line if not performing proposition extraction
    entity_classification_provider=entity_classification_provider # Entity classifications saved to graph between LLM invocations
)

# Create extraction pipeline

extraction_pipeline = ExtractionPipeline.create(
    components=[
        splitter,
        proposition_extractor,
        topic_extractor
        ],

    num_workers=2,
    batch_size=4,
    checkpoint=checkpoint,
    show_progress=True,
    tenant_id=tenant_id
)

# Create build pipeline components

graph_construction = GraphConstruction.for_graph_store(graph_store)
vector_indexing = VectorIndexing.for_vector_store(vector_store)

# Create build pipeline

build_pipeline = BuildPipeline.create(
    components=[
        graph_construction,
        vector_indexing
        ],
    num_workers=2,
    batch_size=10,
    batch_writes_enabled=True,
    checkpoint=checkpoint,
    show_progress=True,
    tenant_id=tenant_id
)

# Load source documents

doc_urls = [

    'https://docs.aws.amazon.com/neptune/latest/userguide/intro.html',

    'https://docs.aws.amazon.com/neptune-analytics/latest/userguide/what-is-neptune-analytics.html',

    'https://docs.aws.amazon.com/neptune-analytics/latest/userguide/neptune-analytics-features.html',

    'https://docs.aws.amazon.com/neptune-analytics/latest/userguide/neptune-analytics-vs-neptune-database.html'

]

docs = SimpleWebPageReader(

    html_to_text=True,

    metadata_fn=lambda url:{'url': url}

).load_data(doc_urls)



# Run the build and exraction stages

docs | extraction_pipeline | build_pipeline | sink


2025-05-06 11:43:41:INFO:g.l.i.e.extraction_pipeline:Running extraction pipeline [batch_size: 4, num_workers: 2]


Extracting propositions [nodes: 5, num_workers: 4]: 100%|██████████| 5/5 [00:11<00:00,  2.32s/it]t]
Extracting propositions [nodes: 10, num_workers: 4]: 100%|██████████| 10/10 [00:20<00:00,  2.04s/it]
Extracting topics [nodes: 10, num_workers: 4]: 100%|██████████| 10/10 [00:49<00:00,  4.99s/it]


2025-05-06 11:44:51:INFO:g.l.i.b.build_pipeline:Running build pipeline [batch_size: 10, num_workers: 2, job_sizes: [441, 183], batch_writes_enabled: True, batch_write_size: 25]


Building graph [batch_writes_enabled: True, batch_write_size: 25]: 100%|██████████| 183/183 [00:00<00:00, 55459.37it/s]
Building graph [batch_writes_enabled: True, batch_write_size: 25]: 100%|██████████| 441/441 [00:00<00:00, 52717.19it/s]
Building vector index [batch_writes_enabled: True, batch_write_size: 25]: 100%|██████████| 183/183 [00:00<00:00, 694622.29it/s]
Building vector index [batch_writes_enabled: True, batch_write_size: 25]: 100%|██████████| 441/441 [00:00<00:00, 1134777.95it/s]


In [ ]:
from graphrag_toolkit.lexical_graph import GraphRAGConfig
print(f"GraphRAGConfig: {GraphRAGConfig}")